### **Docker 使用 `Dockerfile` 與 `docker-compose.yml` 的差異**

### 使用 `docker-compose.yml` 與 `volumes`
當我們使用 `docker-compose.yml` 來管理 Docker 容器時，可以透過 `volumes` 指令來掛載本地的資料夾路徑，使得容器內能夠動態訪問這些路徑。這樣的方式不需要將本地資料夾複製到容器內，具體操作如下：

### 範例 `docker-compose.yml`：
```yaml
version: '3.8'

services:
  fastapi-yolo-app:
    image: fastapi-yolo-app  # Docker 映像名稱
    build: .   
    volumes:
      - "D:/Learning_Python/30-Day_AI_Deep_Learning_Plan/yolov5-master:/app/yolov5-master"  # 將本地 yolov5-master 資料夾掛載到容器內
    ports:
      - "8000:8000"  # 映射主機端口8000到容器端口8000
    tty: true  # 保持容器運行
```

### 優點：
- 你可以保留本地資料夾原位，不必將其複製到與 `Dockerfile` 相同的目錄。
- 本地路徑可以動態同步到容器中，適合需要頻繁修改資料的情境。

## 不使用 `docker-compose.yml`，只使用 `Dockerfile`
如果不使用 `docker-compose.yml`，則無法使用 `volumes` 掛載本地路徑。這種情況下，必須將所有要用到的資料夾與文件放到與 `Dockerfile` 相同的目錄內，這樣才能使用 `COPY` 指令將文件複製到容器內。

### 範例 `Dockerfile`：
```dockerfile
# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
FROM ubuntu:23.10

# 安裝基本依賴
RUN apt update && apt install --no-install-recommends -y \
    python3-pip git zip curl htop libgl1 libglib2.0-0 libpython3-dev gnupg g++ libusb-1.0-0 && \
    rm -rf /var/lib/apt/lists/*

# 升級 pip 並安裝依賴項
COPY requirements.txt .
RUN python3 -m pip install --upgrade pip wheel && \
    pip install --no-cache-dir -r requirements.txt

# 創建工作目錄
RUN mkdir -p /usr/src/app
WORKDIR /usr/src/app

# 複製本地文件夾到容器內
COPY . /usr/src/app
COPY ./yolov5-master /app/yolov5-master  # 必須確保 yolov5-master 與 Dockerfile 位於同一目錄中

# 暴露 FastAPI 使用的端口
EXPOSE 8000

# 啟動 FastAPI 應用
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

### 注意事項：
- **必須**將 `yolov5-master` 資料夾與 `Dockerfile` 放在同一個目錄下，否則 `COPY` 指令無法正常運行。
- 當不使用 `volumes` 掛載時，這是將本地文件夾包含進容器內的唯一方式。

## 總結：
- **使用 `docker-compose.yml` 時**：可以通過 `volumes` 將任何本地資料夾掛載到容器內，不必將文件夾放到與 `Dockerfile` 同一目錄中。
- **不使用 `docker-compose.yml` 時**：所有要複製進容器的文件必須與 `Dockerfile` 位於同一個資料夾內，並使用 `COPY` 指令將它們複製進容器。

---